In [1]:
from espn_api.basketball import League, Matchup, Team
from activity.myUtil import get_opponent_team, get_my_team
from espn_api.basketball.week import Week
from espn_api.basketball.roster_week_predictor import RosterWeekPredictor

from IPython.display import HTML, display
import tabulate

In [2]:
espn_s2 = "SEE README"
swid="SEE README"

In [3]:
league = League(league_id=30695, year=2021, espn_s2=espn_s2, swid=swid)

In [4]:
table_output = [("Team Name", "Team Id")]
[table_output.append((team.team_name, team.team_id)) for team in league.teams]
display(HTML(tabulate.tabulate(table_output, tablefmt='html')))

Team Name,Team Id
CZ ButtKicker,1
SEA MoNeYPro,2
NY Zhanos Diesel,4
Shanghai SHARKS,5
Team Tao,7
Team Beyond,9
PHI Lower Merion 33,11
Badwomen ``,14
Dragon City Tiedan,15
Tim Liang,16


Put your **team id** into the variable below:

In [5]:
MY_TEAM_ID = 2

In [6]:
schedule = league.get_team_data(MY_TEAM_ID).schedule
table_output = [("Week Id", "Opponent Team Name")]
[table_output.append((week_id + 1, get_opponent_team(matchup, MY_TEAM_ID).team_name)) for week_id, matchup in enumerate(schedule)]
display(HTML(tabulate.tabulate(table_output, tablefmt='html')))

Week Id,Opponent Team Name
1,Tim Liang
2,Dragon City Tiedan
3,Team Beyond
4,PHI Lower Merion 33
5,Team Tao
6,CZ ButtKicker
7,NY Zhanos Diesel
8,Badwomen ``
9,Team Tao
10,Tim Liang


In [7]:
WEEK_ID = 3

In [8]:
currWeek = Week(league, WEEK_ID)

In [9]:
predicted_points_team_name_map = {}
number_of_games_team_name_map = {}
for team in league.teams:
    predictor = RosterWeekPredictor(team.roster, currWeek)
    predicted_points_team_name_map[team.team_name] = predictor.predict()
    number_of_games_team_name_map[team.team_name] = predictor.get_total_number_of_games()

In [10]:
def get_tuple_average(tup):
    return (tup[0] + tup[1]) / 2

In [11]:
team_scores = {}
for team in league.teams:
    predicted_points = predicted_points_team_name_map[team.team_name]
    team_scores[team.team_name] = predicted_points[0], predicted_points[1], get_tuple_average(predicted_points)

In [12]:
table_output = []
for team_name, scores in team_scores.items():
    lo, hi, avg = scores
    table_output.append((team_name, number_of_games_team_name_map[team_name], lo, hi, avg))
table_output.sort(reverse=True, key=lambda x: x[-1])

In [13]:
table_output.insert(0, ("Team Name", "# of games", "Low", "High", "Avg"))
display(HTML(tabulate.tabulate(table_output, tablefmt='html')))

Team Name,# of games,Low,High,Avg
Shanghai SHARKS,44,1339.166933066933,1713.9523809523807,1526.5596570096568
CZ ButtKicker,44,1370.4705627705628,1673.6620879120878,1522.0663253413254
Dragon City Tiedan,44,1291.6880952380952,1672.8609307359309,1482.2745129870132
PHI Lower Merion 33,42,1256.642857142857,1555.3203130203133,1405.9815850815853
Tim Liang,42,1302.8570512820513,1505.848901098901,1404.352976190476
NY Zhanos Diesel,41,1268.8269480519484,1539.5690476190475,1404.197997835498
Badwomen ``,38,1186.5192307692305,1428.4556776556778,1307.4874542124542
Team Beyond,40,1139.819130869131,1392.3397435897436,1266.0794372294372
SEA MoNeYPro,36,1064.906227106227,1365.1347985347984,1215.0205128205127
Team Tao,27,811.4499999999999,1022.9653679653679,917.2076839826839


In [14]:
match_up_points = []
match_up_points.append(["Home Team", "Estimate Points", "# of Games", "Away Team", "Estimate Points", "# of Games", "+/-"])
for matchup in league.scoreboard():
    home_team_average = team_scores[matchup.home_team.team_name][-1]
    away_team_average = team_scores[matchup.away_team.team_name][-1]
    match_up_points.append([matchup.home_team.team_name, home_team_average, number_of_games_team_name_map[matchup.home_team.team_name],
                            matchup.away_team.team_name, away_team_average, number_of_games_team_name_map[matchup.away_team.team_name],
                           home_team_average - away_team_average])

In [15]:
display(HTML(tabulate.tabulate(match_up_points, tablefmt='html')))

Home Team,Estimate Points,# of Games,Away Team,Estimate Points,# of Games,+/-
Tim Liang,1404.352976190476,42,NY Zhanos Diesel,1404.197997835498,41,0.1549783549780841
Team Beyond,1266.0794372294372,40,PHI Lower Merion 33,1405.9815850815853,42,-139.90214785214812
Badwomen ``,1307.4874542124542,38,CZ ButtKicker,1522.0663253413254,44,-214.57887112887124
Team Tao,917.2076839826839,27,SEA MoNeYPro,1215.0205128205127,36,-297.8128288378289
Shanghai SHARKS,1526.5596570096568,44,Dragon City Tiedan,1482.2745129870132,44,44.285144022643635


In [16]:
league.get_team_data(MY_TEAM_ID).roster[0].eligibleSlots

['PG', 'G', 'G/F', 'UT', 'BE', 'IR']

In [17]:
from espn_api.basketball.line_up_editer import LineUpEditor

editor = LineUpEditor(league, MY_TEAM_ID)
# editor.bench_all_players(MY_TEAM_ID, 14)

In [18]:
editor.game_day_player_getter.get_active_player_list_for_day(30)

[Player(Trae Young),
 Player(Paul George),
 Player(Pascal Siakam),
 Player(Jonas Valanciunas),
 Player(Victor Oladipo),
 Player(LaMelo Ball),
 Player(Aaron Gordon),
 Player(Wendell Carter Jr.),
 Player(Zach LaVine),
 Player(Collin Sexton),
 Player(Blake Griffin),
 Player(Joe Harris),
 Player(Danny Green),
 Player(Luguentz Dort)]

In [20]:
editor.bench_all_players(33)
editor.fill_line_up(33, ignore_injury=True)

In [22]:
[editor.fill_line_up(period, ignore_injury=True) for period in range(34, 50)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [21]:
[print(entry['playerId'], entry['name']) for entry in line_up['teams'][0]['roster']['entries']]

KeyError: 'name'

In [19]:
from espn_api.basketball.player import Player
players = [(Player(entry)) for entry in line_up['teams'][0]['roster']['entries']]
[print(player.playerId, player.name) for player in league.get_team_data(MY_TEAM_ID).roster]

4277905 Trae Young
4065648 Jayson Tatum
4251 Paul George
3149673 Pascal Siakam
6477 Jonas Valanciunas
2527963 Victor Oladipo
4432816 LaMelo Ball
3064290 Aaron Gordon
4277847 Wendell Carter Jr.
3064440 Zach LaVine
4277811 Collin Sexton
3989 Blake Griffin
2528794 Joe Harris
3988 Danny Green
4397020 Luguentz Dort


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [23]:
Player(line_up['teams'][0]['roster']['entries'][1]).lineUpSlotId

13

In [18]:
league.teams[2].roster[0].lineUpSlotId

3